In [14]:
import os
from PIL import Image


# create three folders original_images, splited_images, and stiched images.
# the first code will split all images in the original folder and new splited images will be saved to splited images fodler.
# the second code will stich all images in the splited images folder and new stiched images will be saved to stiched images folder

def split_all_images_in_folder(input_directory, num_rows, num_cols, output_directory):
    # Create the output directory if it doesn't exist
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)
    
    # Iterate through all files in the input directory
    for filename in os.listdir(input_directory):
        image_path = os.path.join(input_directory, filename)
        if os.path.isfile(image_path) and filename.lower().endswith(('.png', '.jpg', '.jpeg', '.tif')):
            split_image_custom(image_path, num_rows, num_cols, output_directory)
    
    print(f"All images in {input_directory} split and saved to {output_directory}")

def split_image_custom(image_path, num_rows, num_cols, output_directory):
    # Open the image
    image = Image.open(image_path)
    image_width, image_height = image.size
    
    # Calculate the dimensions of each part
    part_width = image_width // num_cols
    part_height = image_height // num_rows
    
    # Split the image
    part_number = 1
    for row in range(num_rows):
        for col in range(num_cols):
            left = col * part_width
            upper = row * part_height
            right = (col + 1) * part_width
            lower = (row + 1) * part_height
            part = image.crop((left, upper, right, lower))
            part_filename = f"{os.path.splitext(os.path.basename(image_path))[0]}_part_{part_number}.png"
            part.save(os.path.join(output_directory, part_filename))
            part_number += 1

    print(f"Image {image_path} split into {num_rows * num_cols} parts and saved to {output_directory}")

# Parameters
input_directory = "/Users/dk/Desktop/dk_unet-270524/split_images/original_images"
output_directory = "/Users/dk/Desktop/dk_unet-270524/split_images/splited_images"
num_rows = 3
num_cols = 2

# Split all images in the folder
split_all_images_in_folder(input_directory, num_rows, num_cols, output_directory)



Image /Users/dk/Desktop/dk_unet-270524/split_images/original_images/Ctr._2h_DFO-Zeitkinetik_4.tif split into 6 parts and saved to /Users/dk/Desktop/dk_unet-270524/split_images/splited_images
All images in /Users/dk/Desktop/dk_unet-270524/split_images/original_images split and saved to /Users/dk/Desktop/dk_unet-270524/split_images/splited_images


In [15]:
import os
from PIL import Image

def stitch_all_images_in_folder(input_directory, num_rows, num_cols, output_directory):
    # Create the output directory if it doesn't exist
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)
    
    # Collect all parts and group them by the base image name
    images_dict = {}
    for filename in os.listdir(input_directory):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.tif')):
            base_name = "_".join(filename.split('_')[:-2])
            if base_name not in images_dict:
                images_dict[base_name] = []
            images_dict[base_name].append(filename)
    
    # Stitch each set of parts back together
    for base_name, parts in images_dict.items():
        parts.sort(key=lambda x: int(x.split('_')[-1].split('.')[0]))  # Sort parts by their number
        stitch_image_custom(base_name, parts, num_rows, num_cols, input_directory, output_directory)

def stitch_image_custom(base_name, parts, num_rows, num_cols, input_directory, output_directory):
    # Read the first part to get the dimensions
    first_part_path = os.path.join(input_directory, parts[0])
    part = Image.open(first_part_path)
    part_width, part_height = part.size
    
    # Create an empty image to hold the stitched result
    stitched_image = Image.new('RGB', (part_width * num_cols, part_height * num_rows))
    
    # Stitch the parts back together
    for index, part_filename in enumerate(parts):
        part_path = os.path.join(input_directory, part_filename)
        part = Image.open(part_path)
        row = index // num_cols
        col = index % num_cols
        stitched_image.paste(part, (col * part_width, row * part_height))
    
    # Save the stitched image
    stitched_image.save(os.path.join(output_directory, f"{base_name}_stitched.png"))
    print(f"Stitched image {base_name} saved to {os.path.join(output_directory, f'{base_name}_stitched.png')}")

# Parameters
input_directory = "/Users/dk/Desktop/dk_unet-270524/split_images/splited_images"
output_directory = "/Users/dk/Desktop/dk_unet-270524/split_images/stiched_images"
num_rows = 3
num_cols = 2

# Stitch all images in the folder
stitch_all_images_in_folder(input_directory, num_rows, num_cols, output_directory)




Stitched image Ctr._2h_DFO-Zeitkinetik_4 saved to /Users/dk/Desktop/dk_unet-270524/split_images/stiched_images/Ctr._2h_DFO-Zeitkinetik_4_stitched.png
